In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
randn = np.random.randn

X = pd.DataFrame(randn(10,3), columns=['X1','X2','X3'])
y = pd.DataFrame(randn(10,1), columns=['Y'])        

model = linear_model.LinearRegression()
model.fit(X=X, y=y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## show scikit-learn results

In [2]:
print("from scikit-learn")
print(model.intercept_)
print(model.coef_)

from scikit-learn
[0.09488465]
[[ 0.83881744  0.0279716  -0.331979  ]]


## reproduce scikit-learn results with linear algebra

In [3]:
N = len(X)
p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

In [4]:
X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = X.values

In [5]:
beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

[[ 0.09488465]
 [ 0.83881744]
 [ 0.0279716 ]
 [-0.331979  ]]


## compute standard errors of the parameter estimates

In [8]:
y_hat = model.predict(X)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

SE(beta_hat[0]): 0.22032357419104698
SE(beta_hat[1]): 0.3864675754487651
SE(beta_hat[2]): 0.2165359527134696
SE(beta_hat[3]): 0.1899559709494176


In [12]:
y.values

array([[-0.44573793],
       [ 1.14812118],
       [-0.27807472],
       [ 1.48267403],
       [-0.06356066],
       [-0.80096423],
       [-0.37930679],
       [-0.90848985],
       [-0.02157307],
       [ 1.25320526]])

## confirm with `statsmodels`

In [8]:
import statsmodels.api as sm
ols = sm.OLS(y.values, X_with_intercept)
ols_result = ols.fit()
print(ols_result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     1.499
Date:                Sat, 12 Oct 2019   Prob (F-statistic):              0.307
Time:                        14:32:46   Log-Likelihood:                -8.7045
No. Observations:                  10   AIC:                             25.41
Df Residuals:                       6   BIC:                             26.62
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2867      0.247     -1.161      0.2